**[Official documentation for OBP](https://zr-obp.readthedocs.io/en/latest/)**

## Quick Reference:

**OBP: Open Bandit Pipeline** - this software library

**OBD: Open Bandit Dataset** - the dataset supplied with it

**OPE: off-policy evaluation** - the process of determining how a policy _other than the one that was really run_ woudl have performed

## Dataset Loader


In [1]:
from pathlib import Path
from obp.dataset import OpenBanditDataset
import numpy as np

DATASET = "../data/obd_full"
# DATASET = "../data/obd"

dataset = OpenBanditDataset(
    data_path=Path(DATASET),
    campaign="all",
    behavior_policy="random"
)

feedback = dataset.obtain_batch_bandit_feedback()

/home/aidanclaffey/anaconda3/envs/bandits-env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Policy and Simulator

Testing out self-implemented `ExploreThenCommit` and comparing with OBP's `Epsilon Greedy`

In [2]:
from obp.policy import EpsilonGreedy

from sd_bandits.obp_extensions.policy import ExploreThenCommit

etc = ExploreThenCommit(
    n_actions=dataset.n_actions,
    len_list = dataset.len_list,
    batch_size=1, # update parameters after every round
    random_state=0,
    min_n = 30
)

eps_greedy = EpsilonGreedy(
    n_actions=dataset.n_actions,
    len_list = dataset.len_list,
    batch_size=1, # update parameters after every round
    random_state=0,
    epsilon=0.2
)

In [3]:
from obp.simulator.simulator import run_bandit_simulation

etc_actions = run_bandit_simulation(bandit_feedback=feedback, policy=etc)

eps_actions = run_bandit_simulation(bandit_feedback=feedback, policy=eps_greedy)

100%|██████████| 1374327/1374327 [00:15<00:00, 89996.37it/s]


How often each action was shown

In [4]:
print('Explore-Then-Commit')
print(etc.action_counts)
print()
print('Epsilon-Greedy')
print(eps_greedy.action_counts)

Explore-Then-Commit
[  30   30   30   30   30   30   30   30   30 1259   30   30   30   30
   30   30   30   30  629   30   30   30   30   30   30   30  315   30
   30   30   30   30   30   30   30   30   30  315   30  315   30   30
   30   30   30 3906   30   30   30   30   30   30   30   30   30   30
   30   30   30   30   30   30   30   30   30   30   30   30   30   30
   30   30   30   30  315  630   30   30   30   30]

Epsilon-Greedy
[1506   60  572   46   49   38   43   67   41   59   41   38   54   46
   42   47   45   41   54   48   38  271   57   45   56   44   35   33
   47   45   49   52   34   44 1891   53   31   51   42 1192   51   56
   47  571   44   39   46   45   34  810   45   42   42   45   42   42
   45   66   39   46   49   40   39   39   43   52   63  147   49   46
   53   47   42   46   38  573   43   41  987 2188]


Relative 'reward' of each action

In [5]:
print('Explore-Then-Commit')
print(etc.reward_counts)
print()
print('Epsilon-Greedy')
print(eps_greedy.reward_counts)

Explore-Then-Commit
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.00317712 0.         0.
 0.         0.         0.         0.         0.         0.
 0.00317965 0.         0.         0.         0.         0.
 0.         0.         0.0031746  0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.0031746  0.         0.0031746  0.         0.
 0.         0.         0.         0.00486431 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.0031746  0.0031746  0.         0.
 0.         0.        ]

Epsilon-Greedy
[0.00464807 0.         0.00174825 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        

How often each action was clicked on

In [6]:
print('Explore-Then-Commit')
print(etc.reward_counts * etc.action_counts)
print()
print('Epsilon-Greedy')
print(eps_greedy.reward_counts * eps_greedy.action_counts)

Explore-Then-Commit
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.
  2.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  1.  0.  0.  0.  0.  0. 19.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  2.  0.  0.  0.  0.]

Epsilon-Greedy
[ 7.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 11.  0.
  0.  0.  0. 19.  0.  0.  0.  1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  3. 18.]


## Off-Policy Evaluation

Some off-polict evaluation methods require a base regression model (like the Direct Method and all methods that use the Direct Method). So, we import LinearRegression in this case, although smarter ones can be used

OBP allows us to measure multiple OPEs at once using the `OffPolicyEvaluation` module

In [7]:
from pathlib import Path
from sklearn.linear_model import LinearRegression
# import open bandit pipeline (obp)
from obp.ope import (
    OffPolicyEvaluation, 
    RegressionModel,
    DirectMethod,
    InverseProbabilityWeighting,
    DoublyRobust,
    DoublyRobustWithShrinkage,
    SelfNormalizedDoublyRobust
)

regression_model = RegressionModel(
    n_actions=dataset.n_actions,
    len_list=dataset.len_list,
    action_context=dataset.action_context,
    base_model=LinearRegression(),
)

estimated_rewards_by_reg_model = regression_model.fit_predict(
    context=feedback["context"],
    action=feedback["action"],
    reward=feedback["reward"],
    position=feedback["position"],
    pscore=feedback["pscore"],
    n_folds=3, # use 3-fold cross-fitting,
    random_state=12345,
)



Explore then commit analysis:

In [8]:
ope = OffPolicyEvaluation(
    bandit_feedback=feedback,
    ope_estimators=[InverseProbabilityWeighting(), DirectMethod(), DoublyRobust(),
                    DoublyRobustWithShrinkage(lambda_ = 1e3), SelfNormalizedDoublyRobust()]
)

# `summarize_off_policy_estimates` returns pandas dataframes including the OPE results
estimated_policy_value, estimated_interval = ope.summarize_off_policy_estimates(
    action_dist=etc_actions, 
    estimated_rewards_by_reg_model=estimated_rewards_by_reg_model,
    random_state=12345,
)

In [9]:
# ground-truth policy value of the random policy
# , which is the empirical mean of the factual (observed) rewards (on-policy estimation)
ground_truth = feedback['reward'].mean()

# calculate estimated policy value of the evaluation policy (the BernoulliTS policy)
# relative to the ground-truth policy value of the behavior policy (the Random policy here)
estimated_policy_value['relative_estimated_policy_value'] =\
        estimated_policy_value.estimated_policy_value / ground_truth

In [10]:
estimated_policy_value

,estimated_policy_value,relative_estimated_policy_value
ipw,0.001805,0.520134
dm,0.003450,0.994374
dr,0.003312,0.954514
dr-os,0.003431,0.988988
sndr,0.005779,1.665754


In [11]:
estimated_interval

,mean,95.0% CI (lower),95.0% CI (upper)
ipw,0.001853,0.001106,0.002650
dm,0.003449,0.003432,0.003457
dr,0.003360,0.002632,0.004153
dr-os,0.003437,0.003340,0.003547
sndr,0.005863,0.004593,0.007248


Visualization for Epsilon-Greedy

In [ ]:
ope.visualize_off_policy_estimates(
    action_dist=eps_actions,
    estimated_rewards_by_reg_model=estimated_rewards_by_reg_model,
    is_relative=True,
    random_state=12345,
)

Visualization for ETC

In [ ]:
ope.visualize_off_policy_estimates(
    action_dist=etc_actions,
    estimated_rewards_by_reg_model=estimated_rewards_by_reg_model,
    is_relative=True,
    random_state=12345,
)